In [15]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import datetime

In [9]:
#obtain list of dates
directory = "Boston Celtics"
list_dates = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        list_dates.append(filename)

In [10]:
# create dictionary 
final_data = {}

In [11]:
opponents = {}
# reads in each csv
for i in list_dates:
    data = pd.read_csv(f"Boston Celtics/{i}", index_col=0)
    #gathers data and adds to dict
    date = data["Date"].tolist()[0]
    
    opponents[date] = data["Opponent"][0]
    
    data[date] = data["Score"]
    final_data[date] = data[date]
    final_data[date].append

In [45]:
# convert dict to dataframe
data_df = pd.DataFrame(final_data)
data_df

,06/16/2022,06/02/2022,06/10/2022,06/05/2022,06/13/2022,04/17/2022,05/25/2022,05/19/2022,05/27/2022,05/23/2022,...,05/13/2022,05/17/2022,05/03/2022,05/29/2022,05/15/2022,05/01/2022,06/08/2022,04/25/2022,04/20/2022,04/23/2022
Aaron Nesmith,0.00,0.00,NaN,11.50,4.25,NaN,0.00,1.25,0.00,10.25,...,1.25,8.75,1.25,NaN,4.75,2.75,1.25,NaN,NaN,NaN
Al Horford,46.00,42.00,24.00,14.50,21.75,44.25,38.25,21.75,20.75,34.25,...,28.50,NaN,39.25,33.50,29.50,36.00,34.50,23.00,34.00,15.25
Daniel Theis,NaN,5.50,NaN,9.25,NaN,13.00,NaN,4.75,NaN,7.75,...,6.50,18.00,NaN,NaN,9.75,6.00,NaN,20.50,26.50,20.00
Derrick White,5.75,28.25,20.75,25.50,8.75,22.25,27.25,NaN,43.25,39.00,...,22.75,10.00,13.50,14.75,9.50,16.50,14.75,19.50,8.25,12.50
Grant Williams,7.25,8.25,4.75,11.25,11.25,13.00,12.50,27.00,6.50,16.25,...,13.25,12.75,33.25,19.50,41.50,16.00,18.25,26.75,30.00,16.75
Jaylen Brown,49.25,45.25,33.50,29.50,34.75,45.25,36.00,40.00,37.50,23.75,...,37.25,44.00,54.50,39.00,33.00,31.25,48.75,38.00,41.00,38.50
Jayson Tatum,33.25,39.25,52.75,43.00,46.00,53.00,49.00,43.25,51.75,51.50,...,66.75,55.50,53.25,54.50,43.50,42.00,49.50,39.25,43.00,65.25
Juwan Morgan,0.00,0.00,NaN,NaN,1.25,NaN,0.00,2.25,NaN,0.00,...,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,NaN
Luke Kornet,0.75,NaN,NaN,3.50,4.75,NaN,0.00,4.50,NaN,NaN,...,NaN,NaN,1.25,NaN,0.00,0.00,NaN,NaN,NaN,NaN
Malik Fitts,NaN,0.00,NaN,3.50,NaN,NaN,0.00,NaN,NaN,0.00,...,NaN,NaN,1.25,NaN,3.50,3.25,0.00,NaN,NaN,NaN


In [46]:
opp = pd.DataFrame.transpose(pd.DataFrame.from_dict(opponents, orient="index"))
data_df = data_df.append(opp)

In [47]:

data_df

,06/16/2022,06/02/2022,06/10/2022,06/05/2022,06/13/2022,04/17/2022,05/25/2022,05/19/2022,05/27/2022,05/23/2022,...,05/13/2022,05/17/2022,05/03/2022,05/29/2022,05/15/2022,05/01/2022,06/08/2022,04/25/2022,04/20/2022,04/23/2022
Aaron Nesmith,0.0,0.0,NaN,11.5,4.25,NaN,0.0,1.25,0.0,10.25,...,1.25,8.75,1.25,NaN,4.75,2.75,1.25,NaN,NaN,NaN
Al Horford,46.0,42.0,24.0,14.5,21.75,44.25,38.25,21.75,20.75,34.25,...,28.5,NaN,39.25,33.5,29.5,36.0,34.5,23.0,34.0,15.25
Daniel Theis,NaN,5.5,NaN,9.25,NaN,13.0,NaN,4.75,NaN,7.75,...,6.5,18.0,NaN,NaN,9.75,6.0,NaN,20.5,26.5,20.0
Derrick White,5.75,28.25,20.75,25.5,8.75,22.25,27.25,NaN,43.25,39.0,...,22.75,10.0,13.5,14.75,9.5,16.5,14.75,19.5,8.25,12.5
Grant Williams,7.25,8.25,4.75,11.25,11.25,13.0,12.5,27.0,6.5,16.25,...,13.25,12.75,33.25,19.5,41.5,16.0,18.25,26.75,30.0,16.75
Jaylen Brown,49.25,45.25,33.5,29.5,34.75,45.25,36.0,40.0,37.5,23.75,...,37.25,44.0,54.5,39.0,33.0,31.25,48.75,38.0,41.0,38.5
Jayson Tatum,33.25,39.25,52.75,43.0,46.0,53.0,49.0,43.25,51.75,51.5,...,66.75,55.5,53.25,54.5,43.5,42.0,49.5,39.25,43.0,65.25
Juwan Morgan,0.0,0.0,NaN,NaN,1.25,NaN,0.0,2.25,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN
Luke Kornet,0.75,NaN,NaN,3.5,4.75,NaN,0.0,4.5,NaN,NaN,...,NaN,NaN,1.25,NaN,0.0,0.0,NaN,NaN,NaN,NaN
Malik Fitts,NaN,0.0,NaN,3.5,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,1.25,NaN,3.5,3.25,0.0,NaN,NaN,NaN


In [53]:
tester["Datetime"] = pd.to_datetime(data_df.columns.tolist()).tolist()

In [49]:
tester = pd.DataFrame.transpose(data_df)
columns = tester.columns.tolist()

In [50]:
#create dict
final_result = {}

In [51]:
tester

,Aaron Nesmith,Al Horford,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser,0
06/16/2022,0.0,46.0,NaN,5.75,7.25,49.25,33.25,0.0,0.75,NaN,33.0,0.0,2.25,30.75,0.0,Golden State Warriors
06/02/2022,0.0,42.0,5.5,28.25,8.25,45.25,39.25,0.0,NaN,0.0,36.25,0.0,19.5,24.5,0.0,Golden State Warriors
06/10/2022,NaN,24.0,NaN,20.75,4.75,33.5,52.75,NaN,NaN,NaN,39.0,NaN,1.75,34.0,NaN,Golden State Warriors
06/05/2022,11.5,14.5,9.25,25.5,11.25,29.5,43.0,NaN,3.5,3.5,11.5,8.75,13.5,10.0,5.0,Golden State Warriors
06/13/2022,4.25,21.75,NaN,8.75,11.25,34.75,46.0,1.25,4.75,NaN,26.25,0.0,0.75,23.0,1.5,Golden State Warriors
04/17/2022,NaN,44.25,13.0,22.25,13.0,45.25,53.0,NaN,NaN,NaN,42.25,NaN,4.75,NaN,NaN,Brooklyn Nets
05/25/2022,0.0,38.25,NaN,27.25,12.5,36.0,49.0,0.0,0.0,0.0,17.25,0.0,1.25,26.5,NaN,Miami Heat
05/19/2022,1.25,21.75,4.75,NaN,27.0,40.0,43.25,2.25,4.5,NaN,63.25,7.75,18.5,17.5,NaN,Miami Heat
05/27/2022,0.0,20.75,NaN,43.25,6.5,37.5,51.75,NaN,NaN,NaN,25.0,NaN,0.0,18.75,NaN,Miami Heat
05/23/2022,10.25,34.25,7.75,39.0,16.25,23.75,51.5,0.0,NaN,0.0,NaN,2.0,26.0,27.25,NaN,Miami Heat


In [44]:
add = {}

#iterate through index
for column in columns:
    print(column)
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence)-1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    rent_rate = list(tester[column].dropna())
    # number of steps
    n_steps = 3
    if len(rent_rate) < 5:
        continue
    X, y = split_sequence(rent_rate, n_steps)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)

    # demonstrate prediction
    x_input = array(rent_rate[-3:])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

Aaron Nesmith
[0.   0.   8.75] 11.5
[ 0.    8.75 11.5 ] 4.25
[ 8.75 11.5   4.25] 0.0
[11.5   4.25  0.  ] 1.25
[4.25 0.   1.25] 0.0
[0.   1.25 0.  ] 10.25
[ 1.25  0.   10.25] 0.0
[ 0.   10.25  0.  ] 1.25
[10.25  0.    1.25] 8.75
[0.   1.25 8.75] 1.25
[1.25 8.75 1.25] 4.75
[8.75 1.25 4.75] 2.75
[1.25 4.75 2.75] 1.25
Al Horford
[46. 42. 25.] 24.0
[42. 25. 24.] 14.5
[25.  24.  14.5] 21.75
[24.   14.5  21.75] 44.25
[14.5  21.75 44.25] 38.25
[21.75 44.25 38.25] 21.75
[44.25 38.25 21.75] 20.75
[38.25 21.75 20.75] 34.25
[21.75 20.75 34.25] 48.0
[20.75 34.25 48.  ] 48.5
[34.25 48.   48.5 ] 34.5
[48.  48.5 34.5] 55.5
[48.5 34.5 55.5] 28.5
[34.5 55.5 28.5] 39.25
[55.5  28.5  39.25] 33.5
[28.5  39.25 33.5 ] 29.5
[39.25 33.5  29.5 ] 36.0
[33.5 29.5 36. ] 34.5
[29.5 36.  34.5] 23.0
[36.  34.5 23. ] 34.0
[34.5 23.  34. ] 15.25
Brodric Thomas
Daniel Theis
[ 5.5  26.75  9.25] 13.0
[26.75  9.25 13.  ] 4.75
[ 9.25 13.    4.75] 7.75
[13.    4.75  7.75] 5.25
[4.75 7.75 5.25] 2.25
[7.75 5.25 2.25] 14.75
[ 5

In [45]:
add

{'Aaron Nesmith': 1.4071203470230103,
 'Al Horford': 16.850967407226562,
 'Daniel Theis': 15.930194854736328,
 'Derrick White': 13.809983253479004,
 'Grant Williams': 25.694034576416016,
 'Jaylen Brown': 40.865169525146484,
 'Jayson Tatum': 50.12757873535156,
 'Juwan Morgan': 0.7012506127357483,
 'Luke Kornet': 0.18349812924861908,
 'Malik Fitts': -0.8474617600440979,
 'Marcus Smart': 28.674304962158203,
 'Nik Stauskas': 0.48079511523246765,
 'Payton Pritchard': 4.313274383544922,
 'Robert Williams III': 25.870563507080078,
 'Sam Hauser': 0.31054550409317017}

In [46]:
# convert dict to dataframe
Data = pd.DataFrame.from_dict(add, orient='index')

In [47]:
Data

,0
Aaron Nesmith,1.407120
Al Horford,16.850967
Daniel Theis,15.930195
Derrick White,13.809983
Grant Williams,25.694035
Jaylen Brown,40.865170
Jayson Tatum,50.127579
Juwan Morgan,0.701251
Luke Kornet,0.183498
Malik Fitts,-0.847462


In [48]:
# convert dataframe to csv
mypath = "Final_Score/LSTM"
if not os.path.isdir(mypath):
    os.makedirs(mypath)
Data.to_csv(f"{mypath}/BostonCeltics.csv", index=True)

Function below used for multiple teams

In [3]:
#obtain list of dates
directorys = ["Boston Celtics"]


for directory in directorys:
    list_dates = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            list_dates.append(filename)
    # create dictionary 
    final_data = {}
    
    # reads in each csv
    for i in list_dates:
        data = pd.read_csv(f"{directory}/{i}", index_col=0)

        #gathers data and adds to dict
        date = data["Date"].tolist()[0]
        data[date] = data["Score"]
        final_data[date] = data[date]
    # convert dict to dataframe
    data_df = pd.DataFrame(final_data)
    
    
    tester = pd.DataFrame.transpose(data_df)
    columns = tester.columns.tolist()
    
    #create dict
    final_result = {}
    
    add = {}

    #iterate through index
    for column in columns:
        print(column)

    # univariate data prep LSTM
        from numpy import array

        # split sequence
        def split_sequence(sequence, n_steps):
            X, y = list(), list()
            for i in range(len(sequence)):
                # find end of pattern 
                end_ix = i + n_steps
                #check if beyond sequence to break
                if end_ix > len(sequence)-1:
                    break
                #gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return array(X), array(y)
        # sequence 
        rent_rate = list(tester[column].dropna())
        # number of steps
        n_steps = 3
        if len(rent_rate) < 5:
            continue
        X, y = split_sequence(rent_rate, n_steps)
        # summarize the data
        for i in range(len(X)):
            print(X[i], y[i])

        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # fit model
        model.fit(X, y, epochs=200, verbose=0)

        # demonstrate prediction
        x_input = array(rent_rate[-3:])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        add[column] = yhat.tolist()[0][0]
    # convert dict to dataframe
    Data = pd.DataFrame.from_dict(add, orient='index')

    
    # convert dataframe to csv
    mypath = f"Final_Score/LSTM/{directory}"
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
    Data.to_csv(f"{mypath}/{directory}.csv", index=True)

Aaron Nesmith
[ 0.   0.  11.5] 4.25
[ 0.   11.5   4.25] 0.0
[11.5   4.25  0.  ] 1.25
[4.25 0.   1.25] 0.0
[0.   1.25 0.  ] 10.25
[ 1.25  0.   10.25] 0.0
[ 0.   10.25  0.  ] 1.25
[10.25  0.    1.25] 8.75
[0.   1.25 8.75] 1.25
[1.25 8.75 1.25] 4.75
[8.75 1.25 4.75] 2.75
[1.25 4.75 2.75] 1.25


2022-08-26 12:00:29.523559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Al Horford
[46. 42. 24.] 14.5
[42.  24.  14.5] 21.75
[24.   14.5  21.75] 44.25
[14.5  21.75 44.25] 38.25
[21.75 44.25 38.25] 21.75
[44.25 38.25 21.75] 20.75
[38.25 21.75 20.75] 34.25
[21.75 20.75 34.25] 48.0
[20.75 34.25 48.  ] 48.5
[34.25 48.   48.5 ] 34.5
[48.  48.5 34.5] 55.5
[48.5 34.5 55.5] 28.5
[34.5 55.5 28.5] 39.25
[55.5  28.5  39.25] 33.5
[28.5  39.25 33.5 ] 29.5
[39.25 33.5  29.5 ] 36.0
[33.5 29.5 36. ] 34.5
[29.5 36.  34.5] 23.0
[36.  34.5 23. ] 34.0
[34.5 23.  34. ] 15.25
Daniel Theis
[ 5.5   9.25 13.  ] 4.75
[ 9.25 13.    4.75] 7.75
[13.    4.75  7.75] 5.25
[4.75 7.75 5.25] 2.25
[7.75 5.25 2.25] 14.75
[ 5.25  2.25 14.75] 0.75
[ 2.25 14.75  0.75] 6.5
[14.75  0.75  6.5 ] 18.0
[ 0.75  6.5  18.  ] 9.75
[ 6.5  18.    9.75] 6.0
[18.    9.75  6.  ] 20.5
[ 9.75  6.   20.5 ] 26.5
[ 6.  20.5 26.5] 20.0
Derrick White
[ 5.75 28.25 20.75] 25.5
[28.25 20.75 25.5 ] 8.75
[20.75 25.5   8.75] 22.25
[25.5   8.75 22.25] 27.25
[ 8.75 22.25 27.25] 43.25
[22.25 27.25 43.25] 39.0
[27.25 43.25 39.

Backtesting

In [29]:
list_dates

['06-16.csv',
 '06-02.csv',
 '06-10.csv',
 '06-05.csv',
 '06-13.csv',
 '04-17.csv',
 '05-25.csv',
 '05-19.csv',
 '05-27.csv',
 '05-23.csv',
 '05-21.csv',
 '05-09.csv',
 '05-11.csv',
 '05-07.csv',
 '05-13.csv',
 '05-17.csv',
 '05-03.csv',
 '05-29.csv',
 '05-15.csv',
 '05-01.csv',
 '06-08.csv',
 '04-25.csv',
 '04-20.csv',
 '04-23.csv']

In [33]:
collect = []
for name in opponents:
    if opponents[name] not in collect:
        collect.append(opponents[name])

In [34]:
collect

['Golden State Warriors', 'Brooklyn Nets', 'Miami Heat', 'Milwaukee Bucks']

In [35]:
data_df

,06/16/2022,06/02/2022,06/10/2022,06/05/2022,06/13/2022,04/17/2022,05/25/2022,05/19/2022,05/27/2022,05/23/2022,...,05/13/2022,05/17/2022,05/03/2022,05/29/2022,05/15/2022,05/01/2022,06/08/2022,04/25/2022,04/20/2022,04/23/2022
Aaron Nesmith,0.00,0.00,NaN,11.50,4.25,NaN,0.00,1.25,0.00,10.25,...,1.25,8.75,1.25,NaN,4.75,2.75,1.25,NaN,NaN,NaN
Al Horford,46.00,42.00,24.00,14.50,21.75,44.25,38.25,21.75,20.75,34.25,...,28.50,NaN,39.25,33.50,29.50,36.00,34.50,23.00,34.00,15.25
Daniel Theis,NaN,5.50,NaN,9.25,NaN,13.00,NaN,4.75,NaN,7.75,...,6.50,18.00,NaN,NaN,9.75,6.00,NaN,20.50,26.50,20.00
Derrick White,5.75,28.25,20.75,25.50,8.75,22.25,27.25,NaN,43.25,39.00,...,22.75,10.00,13.50,14.75,9.50,16.50,14.75,19.50,8.25,12.50
Grant Williams,7.25,8.25,4.75,11.25,11.25,13.00,12.50,27.00,6.50,16.25,...,13.25,12.75,33.25,19.50,41.50,16.00,18.25,26.75,30.00,16.75
Jaylen Brown,49.25,45.25,33.50,29.50,34.75,45.25,36.00,40.00,37.50,23.75,...,37.25,44.00,54.50,39.00,33.00,31.25,48.75,38.00,41.00,38.50
Jayson Tatum,33.25,39.25,52.75,43.00,46.00,53.00,49.00,43.25,51.75,51.50,...,66.75,55.50,53.25,54.50,43.50,42.00,49.50,39.25,43.00,65.25
Juwan Morgan,0.00,0.00,NaN,NaN,1.25,NaN,0.00,2.25,NaN,0.00,...,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,NaN
Luke Kornet,0.75,NaN,NaN,3.50,4.75,NaN,0.00,4.50,NaN,NaN,...,NaN,NaN,1.25,NaN,0.00,0.00,NaN,NaN,NaN,NaN
Malik Fitts,NaN,0.00,NaN,3.50,NaN,NaN,0.00,NaN,NaN,0.00,...,NaN,NaN,1.25,NaN,3.50,3.25,0.00,NaN,NaN,NaN


In [70]:
directorys

['Golden State Warriors', 'Brooklyn Nets', 'Miami Heat', 'Milwaukee Bucks']

In [88]:
#obtain list of dates
directorys = collect


for directory in directorys:
    print(directory)
    list_dates = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            list_dates.append(filename)
    # create dictionary 
    final_data = {}
    
    # reads in each csv
    for i in list_dates:
        data = pd.read_csv(f"{directory}/{i}", index_col=0)

        #gathers data and adds to dict
        date = data["Date"].tolist()[0]
        data[date] = data["Score"]
        final_data[date] = data[date]
    # convert dict to dataframe
    opp = pd.DataFrame(final_data)
    
    
    opp_df = pd.DataFrame.transpose(opp)
    
    opp_df["Datetime"] = pd.to_datetime(opp.columns.tolist()).tolist()
    
    columns = opp_df.columns.tolist()[:-1]
    
    main_test = tester.columns.tolist()[:-2]
    
    for date in tester.loc[tester[0] == directory]["Datetime"]:
        print(date)
        main = tester.loc[tester["Datetime"] < date]
        results = opp_df.loc[opp_df["Datetime"] < date]
    
    
        #create dict
        final_result = {}

        add = {}

        #iterate through index
        for column in columns:
            print(column)

        # univariate data prep LSTM
            from numpy import array

            # split sequence
            def split_sequence(sequence, n_steps):
                X, y = list(), list()
                for i in range(len(sequence)):
                    # find end of pattern 
                    end_ix = i + n_steps
                    #check if beyond sequence to break
                    if end_ix > len(sequence)-1:
                        break
                    #gather input and output parts of the pattern
                    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                    X.append(seq_x)
                    y.append(seq_y)
                return array(X), array(y)
            # sequence 
            rent_rate = list(results[column].dropna())
            # number of steps
            n_steps = 3
            if len(rent_rate) < 5:
                continue
            X, y = split_sequence(rent_rate, n_steps)
            # summarize the data
            for i in range(len(X)):
                print(X[i], y[i])

            # reshape from [samples, timesteps] into [samples, timesteps, features]
            n_features = 1
            X = X.reshape((X.shape[0], X.shape[1], n_features))

            model = Sequential()
            model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')

            # fit model
            model.fit(X, y, epochs=200, verbose=0)

            # demonstrate prediction
            x_input = array(rent_rate[-3:])
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            add[column] = yhat.tolist()[0][0]
        # convert dict to dataframe
        Data = pd.DataFrame.from_dict(add, orient='index')


        # convert dataframe to csv
        mypath = f"Final_Score/LSTM/Backtesting/{directory}"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)
        Data.to_csv(f"{mypath}/{directory}_{date}.csv", index=True)
        
        #create dict
        final_result = {}

        add = {}

        
        #iterate through index
        for column in main_test:
            print(column)

        # univariate data prep LSTM
            from numpy import array

            # split sequence
            def split_sequence(sequence, n_steps):
                X, y = list(), list()
                for i in range(len(sequence)):
                    # find end of pattern 
                    end_ix = i + n_steps
                    #check if beyond sequence to break
                    if end_ix > len(sequence)-1:
                        break
                    #gather input and output parts of the pattern
                    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                    X.append(seq_x)
                    y.append(seq_y)
                return array(X), array(y)
            # sequence 
            rent_rate = list(main[column].dropna())
            # number of steps
            n_steps = 3
            if len(rent_rate) < 5:
                continue
            X, y = split_sequence(rent_rate, n_steps)
            # summarize the data
            for i in range(len(X)):
                print(X[i], y[i])

            # reshape from [samples, timesteps] into [samples, timesteps, features]
            n_features = 1
            X = X.reshape((X.shape[0], X.shape[1], n_features))

            model = Sequential()
            model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')

            # fit model
            model.fit(X, y, epochs=200, verbose=0)

            # demonstrate prediction
            x_input = array(rent_rate[-3:])
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            add[column] = yhat.tolist()[0][0]
        # convert dict to dataframe
        Data = pd.DataFrame.from_dict(add, orient='index')


        # convert dataframe to csv
        mypath = f"Final_Score/LSTM/Backtesting/Boston"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)
        Data.to_csv(f"{mypath}/Boston_{date}.csv", index=True)

Golden State Warriors
2022-06-16 00:00:00
Andre Iguodala
[11.5  3.  10.5] 0.0
[ 3.  10.5  0. ] 0.25
[10.5   0.    0.25] 15.0
Andrew Wiggins
[34.75 42.   21.5 ] 49.25
[42.   21.5  49.25] 32.25
[21.5  49.25 32.25] 34.25
[49.25 32.25 34.25] 21.25
[32.25 34.25 21.25] 35.0
[34.25 21.25 35.  ] 45.75
[21.25 35.   45.75] 36.5
[35.   45.75 36.5 ] 34.75
[45.75 36.5  34.75] 8.25
[36.5  34.75  8.25] 29.25
[34.75  8.25 29.25] 40.25
[ 8.25 29.25 40.25] 33.25
[29.25 40.25 33.25] 28.5
[40.25 33.25 28.5 ] 22.75
[33.25 28.5  22.75] 36.75
[28.5  22.75 36.75] 30.5
[22.75 36.75 30.5 ] 37.0
[36.75 30.5  37.  ] 20.5
Damion Lee
[0. 0. 0.] 4.5
[0.  0.  4.5] 9.25
[0.   4.5  9.25] 15.0
[ 4.5   9.25 15.  ] 0.0
[ 9.25 15.    0.  ] 5.75
[15.    0.    5.75] 1.0
[0.   5.75 1.  ] 14.25
[ 5.75  1.   14.25] 11.75
[ 1.   14.25 11.75] 5.5
[14.25 11.75  5.5 ] 2.75
[11.75  5.5   2.75] 2.25
[5.5  2.75 2.25] 0.0
[2.75 2.25 0.  ] 4.0
Draymond Green
[27.75 32.25 29.25] 26.75
[32.25 29.25 26.75] 39.5
[29.25 26.75 39.5 ] 31.25
[2

Malik Fitts
[0.  3.5 0. ] 0.0
[3.5 0.  0. ] 0.0
[0. 0. 0.] 1.25
[0.   0.   1.25] 3.5
[0.   1.25 3.5 ] 3.25
[1.25 3.5  3.25] 0.0
Marcus Smart
[36.25 39.   11.5 ] 26.25
[39.   11.5  26.25] 42.25
[11.5  26.25 42.25] 17.25
[26.25 42.25 17.25] 63.25
[42.25 17.25 63.25] 25.0
[17.25 63.25 25.  ] 30.0
[63.25 25.   30.  ] 36.0
[25. 30. 36.] 27.5
[30.  36.  27.5] 15.5
[36.  27.5 15.5] 40.25
[27.5  15.5  40.25] 47.75
[15.5  40.25 47.75] 38.25
[40.25 47.75 38.25] 23.25
[47.75 38.25 23.25] 39.25
[38.25 23.25 39.25] 43.75
[23.25 39.25 43.75] 21.25
[39.25 43.75 21.25] 30.75
Nik Stauskas
[0.   8.75 0.  ] 0.0
[8.75 0.   0.  ] 7.75
[0.   0.   7.75] 2.0
[0.   7.75 2.  ] 3.5
[7.75 2.   3.5 ] 2.0
[2.  3.5 2. ] 0.0
[3.5 2.  0. ] 1.5
[2.  0.  1.5] 0.0
[0.  1.5 0. ] 0.0
Payton Pritchard
[19.5   1.75 13.5 ] 0.75
[ 1.75 13.5   0.75] 4.75
[13.5   0.75  4.75] 1.25
[0.75 4.75 1.25] 18.5
[ 4.75  1.25 18.5 ] 0.0
[ 1.25 18.5   0.  ] 26.0
[18.5  0.  26. ] 9.5
[ 0.  26.   9.5] 11.75
[26.    9.5  11.75] 8.0
[ 9.5  11.75

Robert Williams III
[26.5  17.5  18.75] 27.25
[17.5  18.75 27.25] 32.25
[18.75 27.25 32.25] 34.25
[27.25 32.25 34.25] 20.0
[32.25 34.25 20.  ] 6.75
[34.25 20.    6.75] 24.0
[20.    6.75 24.  ] 10.75
[ 6.75 24.   10.75] 8.0
Sam Hauser
2022-06-10 00:00:00
Andre Iguodala
[11.5 10.5  0. ] 0.25
[10.5   0.    0.25] 15.0
Andrew Wiggins
[34.75 21.5  32.25] 34.25
[21.5  32.25 34.25] 21.25
[32.25 34.25 21.25] 35.0
[34.25 21.25 35.  ] 45.75
[21.25 35.   45.75] 36.5
[35.   45.75 36.5 ] 34.75
[45.75 36.5  34.75] 8.25
[36.5  34.75  8.25] 29.25
[34.75  8.25 29.25] 40.25
[ 8.25 29.25 40.25] 33.25
[29.25 40.25 33.25] 28.5
[40.25 33.25 28.5 ] 22.75
[33.25 28.5  22.75] 36.75
[28.5  22.75 36.75] 30.5
[22.75 36.75 30.5 ] 37.0
[36.75 30.5  37.  ] 20.5
Damion Lee
[0.  0.  4.5] 9.25
[0.   4.5  9.25] 15.0
[ 4.5   9.25 15.  ] 0.0
[ 9.25 15.    0.  ] 5.75
[15.    0.    5.75] 1.0
[0.   5.75 1.  ] 14.25
[ 5.75  1.   14.25] 11.75
[ 1.   14.25 11.75] 5.5
[14.25 11.75  5.5 ] 2.75
[11.75  5.5   2.75] 2.25
[5.5  2.75 2

Payton Pritchard
[19.5  13.5   4.75] 1.25
[13.5   4.75  1.25] 18.5
[ 4.75  1.25 18.5 ] 0.0
[ 1.25 18.5   0.  ] 26.0
[18.5  0.  26. ] 9.5
[ 0.  26.   9.5] 11.75
[26.    9.5  11.75] 8.0
[ 9.5  11.75  8.  ] 7.0
[11.75  8.    7.  ] 3.5
[8.  7.  3.5] 33.75
[ 7.    3.5  33.75] 14.75
[ 3.5  33.75 14.75] 0.0
[33.75 14.75  0.  ] 26.75
[14.75  0.   26.75] 9.5
[ 0.   26.75  9.5 ] 4.5
[26.75  9.5   4.5 ] 4.25
[9.5  4.5  4.25] 16.0
[ 4.5   4.25 16.  ] 14.5
Robert Williams III
[24.5 10.  26.5] 17.5
[10.  26.5 17.5] 18.75
[26.5  17.5  18.75] 27.25
[17.5  18.75 27.25] 32.25
[18.75 27.25 32.25] 34.25
[27.25 32.25 34.25] 20.0
[32.25 34.25 20.  ] 6.75
[34.25 20.    6.75] 24.0
[20.    6.75 24.  ] 34.5
[ 6.75 24.   34.5 ] 10.75
[24.   34.5  10.75] 8.0
Sam Hauser
[0.   5.   3.25] 0.0
[5.   3.25 0.  ] 0.0
2022-06-05 00:00:00
Andre Iguodala
Andrew Wiggins
[34.75 32.25 34.25] 21.25
[32.25 34.25 21.25] 35.0
[34.25 21.25 35.  ] 45.75
[21.25 35.   45.75] 36.5
[35.   45.75 36.5 ] 34.75
[45.75 36.5  34.75] 8.25
[36

Sam Hauser
2022-06-13 00:00:00
Andre Iguodala
[11.5 10.5  0. ] 0.25
[10.5   0.    0.25] 15.0
Andrew Wiggins
[34.75 42.   21.5 ] 32.25
[42.   21.5  32.25] 34.25
[21.5  32.25 34.25] 21.25
[32.25 34.25 21.25] 35.0
[34.25 21.25 35.  ] 45.75
[21.25 35.   45.75] 36.5
[35.   45.75 36.5 ] 34.75
[45.75 36.5  34.75] 8.25
[36.5  34.75  8.25] 29.25
[34.75  8.25 29.25] 40.25
[ 8.25 29.25 40.25] 33.25
[29.25 40.25 33.25] 28.5
[40.25 33.25 28.5 ] 22.75
[33.25 28.5  22.75] 36.75
[28.5  22.75 36.75] 30.5
[22.75 36.75 30.5 ] 37.0
[36.75 30.5  37.  ] 20.5
Damion Lee
[0.  0.  4.5] 9.25
[0.   4.5  9.25] 15.0
[ 4.5   9.25 15.  ] 0.0
[ 9.25 15.    0.  ] 5.75
[15.    0.    5.75] 1.0
[0.   5.75 1.  ] 14.25
[ 5.75  1.   14.25] 11.75
[ 1.   14.25 11.75] 5.5
[14.25 11.75  5.5 ] 2.75
[11.75  5.5   2.75] 2.25
[5.5  2.75 2.25] 0.0
[2.75 2.25 0.  ] 4.0
Draymond Green
[27.75 32.25 29.25] 39.5
[32.25 29.25 39.5 ] 31.25
[29.25 39.5  31.25] 30.0
[39.5  31.25 30.  ] 39.0
[31.25 30.   39.  ] 28.25
[30.   39.   28.25] 24.25

...

Nik Stauskas
[7.75 3.5  2.  ] 0.0
[3.5 2.  0. ] 1.5
[2.  0.  1.5] 0.0
Payton Pritchard
[ 4.75 18.5   9.5 ] 11.75
[18.5   9.5  11.75] 8.0
[ 9.5  11.75  8.  ] 7.0
[11.75  8.    7.  ] 3.5
[8.  7.  3.5] 33.75
[ 7.    3.5  33.75] 14.75
[ 3.5  33.75 14.75] 26.75
[33.75 14.75 26.75] 9.5
[14.75 26.75  9.5 ] 4.25
[26.75  9.5   4.25] 16.0
[ 9.5   4.25 16.  ] 14.5
Robert Williams III
[17.5  32.25 34.25] 20.0
[32.25 34.25 20.  ] 24.0
[34.25 20.   24.  ] 10.75
[20.   24.   10.75] 8.0
Sam Hauser
2022-05-21 00:00:00
Bam Adebayo
[23.   19.75 35.25] 25.5
[19.75 35.25 25.5 ] 40.25
[35.25 25.5  40.25] 24.5
[25.5  40.25 24.5 ] 16.75
[40.25 24.5  16.75] 23.5
[24.5  16.75 23.5 ] 49.5
[16.75 23.5  49.5 ] 42.25
[23.5  49.5  42.25] 18.5
[49.5  42.25 18.5 ] 25.0
[42.25 18.5  25.  ] 38.75
Caleb Martin
[ 5.25 10.    0.  ] 8.5
[10.   0.   8.5] 11.0
[ 0.   8.5 11. ] 5.25
[ 8.5  11.    5.25] 1.25
[11.    5.25  1.25] 6.75
[5.25 1.25 6.75] 13.25
[ 1.25  6.75 13.25] 16.0
[ 6.75 13.25 16.  ] 9.5
[13.25 16.    9.5 ] 7.0


Derrick White
[22.25 23.   28.25] 19.5
[23.   28.25 19.5 ] 22.75
[28.25 19.5  22.75] 13.5
[19.5  22.75 13.5 ] 9.5
[22.75 13.5   9.5 ] 16.5
[13.5  9.5 16.5] 19.5
[ 9.5 16.5 19.5] 8.25
[16.5  19.5   8.25] 12.5
Grant Williams
[13.  13.5  3. ] 25.5
[13.5  3.  25.5] 13.25
[ 3.   25.5  13.25] 33.25
[25.5  13.25 33.25] 41.5
[13.25 33.25 41.5 ] 16.0
[33.25 41.5  16.  ] 26.75
[41.5  16.   26.75] 30.0
[16.   26.75 30.  ] 16.75
Jaylen Brown
[45.25 25.25 47.5 ] 48.5
[25.25 47.5  48.5 ] 37.25
[47.5  48.5  37.25] 54.5
[48.5  37.25 54.5 ] 33.0
[37.25 54.5  33.  ] 31.25
[54.5  33.   31.25] 38.0
[33.   31.25 38.  ] 41.0
[31.25 38.   41.  ] 38.5
Jayson Tatum
[53.   54.25 52.  ] 24.25
[54.25 52.   24.25] 66.75
[52.   24.25 66.75] 53.25
[24.25 66.75 53.25] 43.5
[66.75 53.25 43.5 ] 42.0
[53.25 43.5  42.  ] 39.25
[43.5  42.   39.25] 43.0
[42.   39.25 43.  ] 65.25
Juwan Morgan
Luke Kornet
Malik Fitts
Marcus Smart
[42.25 36.   27.5 ] 15.5
[36.  27.5 15.5] 40.25
[27.5  15.5  40.25] 38.25
[15.5  40.25 38.25] 23

Robert Williams III
[26.5  17.5  18.75] 27.25
[17.5  18.75 27.25] 32.25
[18.75 27.25 32.25] 34.25
[27.25 32.25 34.25] 20.0
[32.25 34.25 20.  ] 24.0
[34.25 20.   24.  ] 10.75
[20.   24.   10.75] 8.0
Sam Hauser
Milwaukee Bucks
2022-05-09 00:00:00
Bobby Portis
[33.   22.75 22.  ] 32.75
[22.75 22.   32.75] 36.25
[22.   32.75 36.25] 27.0
[32.75 36.25 27.  ] 4.0
[36.25 27.    4.  ] 39.5
Brook Lopez
[27.75 32.   15.  ] 26.5
[32.  15.  26.5] 21.75
[15.   26.5  21.75] 16.25
[26.5  21.75 16.25] 36.25
[21.75 16.25 36.25] 17.5
George Hill
Giannis Antetokounmpo
[53.5  76.   51.25] 59.75
[76.   51.25 59.75] 47.75
[51.25 59.75 47.75] 66.25
[59.75 47.75 66.25] 73.5
[47.75 66.25 73.5 ] 40.25
Grayson Allen
[ 5.25 10.5  11.  ] 22.0
[10.5 11.  22. ] 23.25
[11.   22.   23.25] 38.5
[22.   23.25 38.5 ] 5.75
[23.25 38.5   5.75] 36.0
Jevon Carter
[24.5   6.    1.25] 5.25
[6.   1.25 5.25] 10.25
[ 1.25  5.25 10.25] 25.75
[ 5.25 10.25 25.75] 9.0
[10.25 25.75  9.  ] 2.5
[25.75  9.    2.5 ] 9.25
Jordan Nwora
[45.  

George Hill
Giannis Antetokounmpo
[53.5  62.   59.75] 76.0
[62.   59.75 76.  ] 51.25
[59.75 76.   51.25] 59.75
[76.   51.25 59.75] 47.75
[51.25 59.75 47.75] 66.25
[59.75 47.75 66.25] 73.5
[47.75 66.25 73.5 ] 40.25
Grayson Allen
[ 5.25 11.5  19.  ] 10.5
[11.5 19.  10.5] 11.0
[19.  10.5 11. ] 22.0
[10.5 11.  22. ] 23.25
[11.   22.   23.25] 38.5
[22.   23.25 38.5 ] 5.75
[23.25 38.5   5.75] 36.0
Jevon Carter
[24.5  6.   0. ] 1.25
[6.   0.   1.25] 5.25
[0.   1.25 5.25] 10.25
[ 1.25  5.25 10.25] 25.75
[ 5.25 10.25 25.75] 9.0
[10.25 25.75  9.  ] 2.5
[25.75  9.    2.5 ] 9.25
Jordan Nwora
[45.    0.    1.25] 1.25
[0.   1.25 1.25] 0.0
[1.25 1.25 0.  ] 1.5
[1.25 0.   1.5 ] 4.75
Jrue Holiday
[ 0.   34.   43.75] 53.0
[34.   43.75 53.  ] 46.25
[43.75 53.   46.25] 37.5
[53.   46.25 37.5 ] 50.25
[46.25 37.5  50.25] 27.25
[37.5  50.25 27.25] 42.5
[50.25 27.25 42.5 ] 31.5
[27.25 42.5  31.5 ] 36.0
Khris Middleton
Lindell Wigginton
Luca Vildoza
[0. 0. 0.] 0.0
[0. 0. 0.] 13.25
Pat Connaughton
[10.   25.25 

Nik Stauskas
Payton Pritchard
[ 4.75 11.75  8.  ] 7.0
[11.75  8.    7.  ] 3.5
[8.  7.  3.5] 14.75
[ 7.    3.5  14.75] 9.5
[ 3.5  14.75  9.5 ] 4.25
[14.75  9.5   4.25] 16.0
[ 9.5   4.25 16.  ] 14.5
Robert Williams III
[32.25 20.   24.  ] 10.75
[20.   24.   10.75] 8.0
Sam Hauser
2022-05-01 00:00:00
Bobby Portis
[33.   36.25 27.  ] 4.0
[36.25 27.    4.  ] 39.5
Brook Lopez
[27.75 21.75 16.25] 36.25
[21.75 16.25 36.25] 17.5
George Hill
Giannis Antetokounmpo
[53.5  47.75 66.25] 73.5
[47.75 66.25 73.5 ] 40.25
Grayson Allen
[ 5.25 23.25 38.5 ] 5.75
[23.25 38.5   5.75] 36.0
Jevon Carter
[24.5   6.   25.75] 9.0
[ 6.   25.75  9.  ] 2.5
[25.75  9.    2.5 ] 9.25
Jordan Nwora
Jrue Holiday
[ 0.   34.   27.25] 42.5
[34.   27.25 42.5 ] 31.5
[27.25 42.5  31.5 ] 36.0
Khris Middleton
Lindell Wigginton
Luca Vildoza
Pat Connaughton
[10.  26.5 16. ] 10.25
[26.5  16.   10.25] 22.5
Rayjon Tucker
Sandro Mamukelashvili
Serge Ibaka
Thanasis Antetokounmpo
Wesley Matthews
[13.75 24.25 12.25] 20.5
[24.25 12.25 20.5 

In [87]:
for i in tester.loc[tester[0] == directory]["Datetime"]:
    print(i)

2022-05-09 00:00:00
2022-05-11 00:00:00
2022-05-07 00:00:00
2022-05-13 00:00:00
2022-05-03 00:00:00
2022-05-15 00:00:00
2022-05-01 00:00:00


In [86]:
tester

,Aaron Nesmith,Al Horford,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser,0,Datetime
06/16/2022,0.0,46.0,NaN,5.75,7.25,49.25,33.25,0.0,0.75,NaN,33.0,0.0,2.25,30.75,0.0,Golden State Warriors,2022-06-16
06/02/2022,0.0,42.0,5.5,28.25,8.25,45.25,39.25,0.0,NaN,0.0,36.25,0.0,19.5,24.5,0.0,Golden State Warriors,2022-06-02
06/10/2022,NaN,24.0,NaN,20.75,4.75,33.5,52.75,NaN,NaN,NaN,39.0,NaN,1.75,34.0,NaN,Golden State Warriors,2022-06-10
06/05/2022,11.5,14.5,9.25,25.5,11.25,29.5,43.0,NaN,3.5,3.5,11.5,8.75,13.5,10.0,5.0,Golden State Warriors,2022-06-05
06/13/2022,4.25,21.75,NaN,8.75,11.25,34.75,46.0,1.25,4.75,NaN,26.25,0.0,0.75,23.0,1.5,Golden State Warriors,2022-06-13
04/17/2022,NaN,44.25,13.0,22.25,13.0,45.25,53.0,NaN,NaN,NaN,42.25,NaN,4.75,NaN,NaN,Brooklyn Nets,2022-04-17
05/25/2022,0.0,38.25,NaN,27.25,12.5,36.0,49.0,0.0,0.0,0.0,17.25,0.0,1.25,26.5,NaN,Miami Heat,2022-05-25
05/19/2022,1.25,21.75,4.75,NaN,27.0,40.0,43.25,2.25,4.5,NaN,63.25,7.75,18.5,17.5,NaN,Miami Heat,2022-05-19
05/27/2022,0.0,20.75,NaN,43.25,6.5,37.5,51.75,NaN,NaN,NaN,25.0,NaN,0.0,18.75,NaN,Miami Heat,2022-05-27
05/23/2022,10.25,34.25,7.75,39.0,16.25,23.75,51.5,0.0,NaN,0.0,NaN,2.0,26.0,27.25,NaN,Miami Heat,2022-05-23


In [80]:
main

,Aaron Nesmith,Al Horford,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser,0,Datetime
06/02/2022,0.0,42.0,5.5,28.25,8.25,45.25,39.25,0.0,NaN,0.0,36.25,0.0,19.5,24.5,0.0,Golden State Warriors,2022-06-02
06/10/2022,NaN,24.0,NaN,20.75,4.75,33.5,52.75,NaN,NaN,NaN,39.0,NaN,1.75,34.0,NaN,Golden State Warriors,2022-06-10
06/05/2022,11.5,14.5,9.25,25.5,11.25,29.5,43.0,NaN,3.5,3.5,11.5,8.75,13.5,10.0,5.0,Golden State Warriors,2022-06-05
06/13/2022,4.25,21.75,NaN,8.75,11.25,34.75,46.0,1.25,4.75,NaN,26.25,0.0,0.75,23.0,1.5,Golden State Warriors,2022-06-13
04/17/2022,NaN,44.25,13.0,22.25,13.0,45.25,53.0,NaN,NaN,NaN,42.25,NaN,4.75,NaN,NaN,Brooklyn Nets,2022-04-17
05/25/2022,0.0,38.25,NaN,27.25,12.5,36.0,49.0,0.0,0.0,0.0,17.25,0.0,1.25,26.5,NaN,Miami Heat,2022-05-25
05/19/2022,1.25,21.75,4.75,NaN,27.0,40.0,43.25,2.25,4.5,NaN,63.25,7.75,18.5,17.5,NaN,Miami Heat,2022-05-19
05/27/2022,0.0,20.75,NaN,43.25,6.5,37.5,51.75,NaN,NaN,NaN,25.0,NaN,0.0,18.75,NaN,Miami Heat,2022-05-27
05/23/2022,10.25,34.25,7.75,39.0,16.25,23.75,51.5,0.0,NaN,0.0,NaN,2.0,26.0,27.25,NaN,Miami Heat,2022-05-23
05/21/2022,0.0,48.0,5.25,8.0,20.75,54.75,21.0,1.25,0.0,0.0,30.0,3.5,9.5,NaN,NaN,Miami Heat,2022-05-21


In [74]:
tester.columns.tolist()[:-2]

['Aaron Nesmith',
 'Al Horford',
 'Daniel Theis',
 'Derrick White',
 'Grant Williams',
 'Jaylen Brown',
 'Jayson Tatum',
 'Juwan Morgan',
 'Luke Kornet',
 'Malik Fitts',
 'Marcus Smart',
 'Nik Stauskas',
 'Payton Pritchard',
 'Robert Williams III',
 'Sam Hauser']

In [40]:
tester.index.tolist()[0]

'06/16/2022'

In [62]:
for i in tester.loc[tester[0] == 'Golden State Warriors']["Datetime"]:
    t

2022-06-16 00:00:00
2022-06-02 00:00:00
2022-06-10 00:00:00
2022-06-05 00:00:00
2022-06-13 00:00:00
2022-06-08 00:00:00


In [64]:
tester.loc[tester["Datetime"] < i]

,Aaron Nesmith,Al Horford,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser,0,Datetime
06/02/2022,0.0,42.0,5.5,28.25,8.25,45.25,39.25,0.0,NaN,0.0,36.25,0.0,19.5,24.5,0.0,Golden State Warriors,2022-06-02
06/05/2022,11.5,14.5,9.25,25.5,11.25,29.5,43.0,NaN,3.5,3.5,11.5,8.75,13.5,10.0,5.0,Golden State Warriors,2022-06-05
04/17/2022,NaN,44.25,13.0,22.25,13.0,45.25,53.0,NaN,NaN,NaN,42.25,NaN,4.75,NaN,NaN,Brooklyn Nets,2022-04-17
05/25/2022,0.0,38.25,NaN,27.25,12.5,36.0,49.0,0.0,0.0,0.0,17.25,0.0,1.25,26.5,NaN,Miami Heat,2022-05-25
05/19/2022,1.25,21.75,4.75,NaN,27.0,40.0,43.25,2.25,4.5,NaN,63.25,7.75,18.5,17.5,NaN,Miami Heat,2022-05-19
05/27/2022,0.0,20.75,NaN,43.25,6.5,37.5,51.75,NaN,NaN,NaN,25.0,NaN,0.0,18.75,NaN,Miami Heat,2022-05-27
05/23/2022,10.25,34.25,7.75,39.0,16.25,23.75,51.5,0.0,NaN,0.0,NaN,2.0,26.0,27.25,NaN,Miami Heat,2022-05-23
05/21/2022,0.0,48.0,5.25,8.0,20.75,54.75,21.0,1.25,0.0,0.0,30.0,3.5,9.5,NaN,NaN,Miami Heat,2022-05-21
05/09/2022,NaN,48.5,2.25,23.0,13.5,25.25,54.25,NaN,NaN,NaN,36.0,NaN,11.75,NaN,NaN,Milwaukee Bucks,2022-05-09
05/11/2022,NaN,34.5,14.75,28.25,3.0,47.5,52.0,NaN,NaN,NaN,27.5,NaN,8.0,NaN,NaN,Milwaukee Bucks,2022-05-11
